# Clustering Attempt SEIRS

In [1]:
import numpy as np
from ot import emd2
from sklearn.cluster import KMeans
from scipy import stats
from scipy import integrate
import geopandas as gpd
import folium

2023-01-11 09:58:51.099669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 09:58:51.198748: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-11 09:58:51.201926: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dualta/Documents/soft/aips/31DEC22/LNX64/LIBR/INTELCMP
2023-01-11 09:58:51.20

In [2]:
# Get NDVI data for control and treatment zones
tzo = np.load('treatment_zone_observed.pkl', allow_pickle=True)
czo = np.load('control_zone_observed.pkl', allow_pickle=True)

In [21]:
# Irrigation shapefiles for the region
shp = gpd.read_file('/home/dualta/Documents/tapas/seirs_data/shapefiles/irrigation_shapefile_africarice/SRV_irr_v10/SRV_irr_v10.shp')
shp = shp[shp['Country_1'] == 'SN']

In [4]:
# Turn NDVI location data into the correct CRS projection
from fiona.crs import from_epsg 
from shapely import Point

tzo['geometry'].crs = "EPSG:4326"
czo['geometry'].crs = "EPSG:4326"

tzo = tzo.to_crs(crs=shp.crs)
czo = czo.to_crs(crs=shp.crs)

In [57]:
matches = gpd.sjoin(czo, shp, how='left', op='within')

/home/dualta/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [88]:
polygon_matches = shp[shp.index.isin(list(matches['index_right'].dropna().unique().astype(int)))]